### position vector

In [1]:
# from sympy import symbols
from sympy.vector import CoordSys3D, VectorAdd
import sympy as sp

In [2]:
def initialize(old_symbols, cartesian_position_formula):
    C = CoordSys3D('C')  # standard Cartesian system
    t = sp.symbols('t')

    # make sure our symbols become (undefined) functions of t
    def func_of_t(*args, **kwargs):
        return sp.Function(*args, **kwargs)(t)
    U = sp.symbols(' '.join(s.name for s in old_symbols), cls=func_of_t)
    symbol_substitution = list(zip(old_symbols, U))

    position_vector: VectorAdd = cartesian_position_formula[0]*C.i + cartesian_position_formula[1]*C.j + cartesian_position_formula[2]*C.k
    position_vector = position_vector.subs(symbol_substitution)
    return t, U, symbol_substitution, position_vector

### get direction vectors of each component (define lamé coeffs and r_hat, theta_hat etc.)

In [3]:
def unit_vectors(position_vector, U):
    unit_vectors_map = {u: sp.simplify(sp.diff(position_vector, u).normalize()) for u in U}
    return unit_vectors_map

#### version which prints prettier equations (via given_lame) - ickier code

def unit_vectors_pretty(position_vector, U, given_lame, symbol_substitution):
    lame = {}
    norm = lambda v: sp.sqrt(v.dot(v))
    calculate_lame = lambda u: sp.simplify(norm(sp.diff(r_, u)))
    for old_u, u in symbol_substitution:
        given_lame_u = given_lame.get(old_u)
        if given_lame_u:
            lame[u] = sp.simplify(given_lame_u).subs(symbol_substitution)
        else:
            lame[u] = calculate_lame(u)

    # derivatives of position vectors wrt time
    # we COULD use .normal() here but this has UGLY lames gr
    unit_vectors_map = {u: sp.diff(position_vector, u)/lame[u] for u in U}
    return unit_vectors_map

### acceleration

In [4]:
def get_acceleration_2(a_formula, t, U, unit_vectors_map):
    a_in_direction = lambda u: sp.simplify(a_formula.dot(unit_vectors_map[u]))

    F = {u: sp.symbols(f'F_{u.name}') for u in U}
    U_acceleration = {}
    for u in U:
        # get acceleration in terms of u
        u_a_solns = sp.solve(sp.Eq(a_in_direction(u), F[u]), sp.diff(u, t, 2))
        if len(u_a_solns) != 1:
            raise ValueError(f'could not find acceleration in direction of {u} in terms of {sp.diff(u, t, 2)}.\nreturned solns: {u_a_solns}')
        U_acceleration[u] = u_a_solns[0]

    return U_acceleration

In [5]:
def get_acceleration_vector(a_formula, t, U, unit_vectors_map):
    a_in_direction = lambda u: a_formula.dot(unit_vectors_map[u])
    U_acceleration = {u: a_in_direction(u) for u in U}
    return U_acceleration

### Combining these:

In [6]:
def get_coords_1(old_symbols, cartesian_position_formula, given_lame=None):
    t, U, symbol_substitution, position_vector = initialize(old_symbols, cartesian_position_formula)
    if given_lame:
        unit_vectors_map = unit_vectors_pretty(position_vector, U, given_lame, symbol_substitution)
    else:
        unit_vectors_map = unit_vectors(position_vector, U)

    ## Specific to system
    # a_formula = sp.diff(U[0]*unit_vectors_map[U[0]], t, 2)
    ## Generally:
    a_formula = sp.diff(position_vector, t, 2)
    
    # acceleration_0 = get_acceleration_vector(a_formula, t, U, unit_vectors_map)
    # # for val in acceleration_0.values():
    # #     display(val)

    acceleration = get_acceleration_2(a_formula, t, U, unit_vectors_map)

    return acceleration

### Spherical

In [7]:
def caller(specify_lame=False):
    from sympy import sin, cos
    r, θ, φ = sp.symbols('r θ φ')
    old_symbols = (r, θ, φ)
    position_vector = ((r*sin(θ)*cos(φ)), (r*sin(θ)*sin(φ)), (-r*cos(θ)))

    given_lame = {
        r: 1,
        θ: r,
        ϕ: r*sin(θ)
    }

    if specify_lame:
        acceleration = get_coords_1(old_symbols, position_vector, given_lame)
    else:
        acceleration = get_coords_1(old_symbols, position_vector)

    for u_a in acceleration.values():
        display(sp.simplify(u_a))

In [8]:
caller(specify_lame=False)

F_r + r(t)*sin(θ(t))**2*Derivative(φ(t), t)**2 + r(t)*Derivative(θ(t), t)**2

F_θ*sqrt(r(t)**2)/r(t)**2 + sin(2*θ(t))*Derivative(φ(t), t)**2/2 - 2*Derivative(r(t), t)*Derivative(θ(t), t)/r(t)

F_φ*sqrt(r(t)**2*sin(θ(t))**2)/(r(t)**2*sin(θ(t))**2) - 2*Derivative(θ(t), t)*Derivative(φ(t), t)/tan(θ(t)) - 2*Derivative(r(t), t)*Derivative(φ(t), t)/r(t)

In [9]:
caller(specify_lame=True)

F_r + r(t)*sin(θ(t))**2*Derivative(φ(t), t)**2 + r(t)*Derivative(θ(t), t)**2

(F_θ + r(t)*sin(2*θ(t))*Derivative(φ(t), t)**2/2 - 2*Derivative(r(t), t)*Derivative(θ(t), t))/r(t)

(F_φ/sin(θ(t)) - 2*r(t)*Derivative(θ(t), t)*Derivative(φ(t), t)/tan(θ(t)) - 2*Derivative(r(t), t)*Derivative(φ(t), t))/r(t)

in my opinion, this all looks fine without specifying the lamé coefficients

I don't really see what it would change in a practical sense tbh

### Other systems of coordinates?
(I'm scared about this one just in case I ignored smth important)

NB: we have required an **orthonormal** system...

#### Cartesian:

In [10]:
def caller_cartesian(specify_lame=False):
    from sympy import sin, cos
    x, y, z = sp.symbols('x y z')
    old_symbols = (x, y, z)
    position_vector = (x, y, z)

    acceleration = get_coords_1(old_symbols, position_vector)
    for u_a in acceleration.values():
        display(sp.simplify(u_a))

caller_cartesian()

F_x

F_y

F_z

#### Cylindrical

In [11]:
def caller_cylindrical(specify_lame=False):
    from sympy import sin, cos
    r, θ, z = sp.symbols('r θ z')
    old_symbols = (r, θ, z)
    position_vector = (r*cos(θ), r*sin(θ), z)

    given_lame = {
        r: 1,
        θ: r,
        z: 1
    }

    if specify_lame:
        acceleration = get_coords_1(old_symbols, position_vector, given_lame)
    else:
        acceleration = get_coords_1(old_symbols, position_vector)

    for u_a in acceleration.values():
        display(sp.simplify(u_a))

caller_cylindrical(specify_lame=True)
print('\n\n')
caller_cylindrical(specify_lame=False)

F_r + r(t)*Derivative(θ(t), t)**2

(F_θ - 2*Derivative(r(t), t)*Derivative(θ(t), t))/r(t)

F_z

F_r + r(t)*Derivative(θ(t), t)**2

(F_θ*sqrt(r(t)**2) - 2*r(t)*Derivative(r(t), t)*Derivative(θ(t), t))/r(t)**2

F_z